In [4]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
!pip install keras-tcn
from tcn import TCN

In [48]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset2/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset2/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset2/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset2/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [49]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [50]:
input_shape = x_train.shape[1:]
inp = Input(shape=x_train.shape[1:])
x = SpatialDropout1D(0.2)(inp)
x = TCN(128,dilations = [1, 2, 4], return_sequences=True)(x)
x = TCN(64,dilations = [1, 2, 4], return_sequences=True)(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
    
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.2)(conc)
outp = Dense(20, activation="softmax")(conc)    
model = Model(inputs=inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 128, 6)]     0           []                               
                                                                                                  
 spatial_dropout1d_10 (SpatialD  (None, 128, 6)      0           ['input_11[0][0]']               
 ropout1D)                                                                                        
                                                                                                  
 tcn_20 (TCN)                   (None, 128, 128)     249728      ['spatial_dropout1d_10[0][0]']   
                                                                                                  
 tcn_21 (TCN)                   (None, 128, 64)      94656       ['tcn_20[0][0]']          

In [51]:
history = model.fit(x_train, y_train, epochs=50, verbose=True, validation_data=(x_validation, y_validation), batch_size=128)

Epoch 1/50
278/278 [==============================] - 24s 76ms/step - loss: 1.5468 - accuracy: 0.6103 - val_loss: 0.2491 - val_accuracy: 0.9325
Epoch 2/50
278/278 [==============================] - 21s 74ms/step - loss: 0.4742 - accuracy: 0.8614 - val_loss: 0.1540 - val_accuracy: 0.9575
Epoch 3/50
278/278 [==============================] - 20s 73ms/step - loss: 0.3191 - accuracy: 0.9096 - val_loss: 0.1131 - val_accuracy: 0.9708
Epoch 4/50
278/278 [==============================] - 20s 74ms/step - loss: 0.2623 - accuracy: 0.9250 - val_loss: 0.0898 - val_accuracy: 0.9759
Epoch 5/50
278/278 [==============================] - 20s 74ms/step - loss: 0.2259 - accuracy: 0.9355 - val_loss: 0.0771 - val_accuracy: 0.9805
Epoch 6/50
278/278 [==============================] - 20s 74ms/step - loss: 0.1963 - accuracy: 0.9445 - val_loss: 0.0796 - val_accuracy: 0.9797
Epoch 7/50
278/278 [==============================] - 20s 73ms/step - loss: 0.1833 - accuracy: 0.9484 - val_loss: 0.0713 - val_accuracy:

In [52]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9912
Testing Accuracy: 0.9615
